# Test Best Models

Test notebook to generate the statistics of the different models found with AMS. First load the required libraries

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
from keras.models import model_from_json

from sklearn.model_selection import KFold
from scipy import stats

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler(data_scaler=None):

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features,
                                       max_rul, window_size, window_stride, data_scaler=data_scaler)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps

### Load models and evaluate model

In [4]:
def load_model(model_file, weights_file="", problem_type=1):
    
    p_type = ""
    
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("Loaded model from disk") 
        
    #Load weights if specified
    if weights_file != "":
        # load weights into new model
        loaded_model.load_weights(weights_file)
        print("Loaded weights from disk") 
    else:
        print("Model needs training")
        
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        p_type = "regression"
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        p_type = "classification"
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    print("Created model for " + p_type + " with loss function " + lossFunction)

    return loaded_model
        
        

### Load each of the models and test them

In [5]:
best_model_folder = 'best_models'
"""
test_sets = {'cifar10':(CIFAR10DataHandler, None, 2), 
             'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1), 
             'mnist':((MNISTDataHandler), None, 2)}
"""

test_sets = {'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1)}
alpha_folders = ['alpha0.6', 'alpha0.8', 'alpha1']
#alpha_folders = ['alpha1']

In [6]:
results = {}

k = 10  #For 10-fold cross validation

model_file_name = 'bestModel_global.json'
weights_file_name = 'bestModel_global.h5'

for dataset in test_sets.keys():
    for alphaValue in alpha_folders:
        
        results_key = dataset + "_" +  alphaValue
        
        evaluations_cv = np.zeros(k)
        evaluations_test = np.zeros(k)
        
        model_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + model_file_name
        
        if weights_file_name != "":
            weights_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + weights_file_name
        
        dhandler, data_scaler, problem_type = test_sets[dataset]

        #model = load_model(model_location, weights_location, problem_type)   
        
        data_handler = dhandler(data_scaler=data_scaler)
        data_handler.load_data(verbose = 1, )
        data_handler.print_data()
        
        folds = list(KFold(n_splits=k, shuffle=True).split(data_handler.X_train))
        
        print('Validation on model:' + model_location)
        
        for j, (train_idx, val_idx) in enumerate(folds):

            print('\nExperiment on Fold ', j)
            
            K.clear_session()  #Clear the previous tensorflow graph 

            X_train_cv = data_handler.X_train[train_idx]
            y_train_cv = data_handler.y_train[train_idx]
            X_valid_cv = data_handler.X_train[val_idx]
            y_valid_cv = data_handler.y_train[val_idx]

            model = load_model(model_location, "", problem_type)
            model.summary()

            model.fit(X_train_cv, y_train_cv, batch_size=512, epochs=100, verbose=1)

            evaluation_cv = model.evaluate(X_valid_cv, y_valid_cv)
            evaluation_test = model.evaluate(data_handler.X_test, data_handler.y_test)

            evaluations_cv[j] = evaluation_cv[1]
            evaluations_test[j] = evaluation_test[1]
            
        results[results_key] = (evaluations_cv, evaluations_test)
        

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ... -0.33333333  0.33333333
   0.31289699]
 [-0.43373494 -0.09396119 -0.29473329 ... -0.16666667  0.25581395
   0.47638774]
 [-0.31325301 -0.26095487 -0.25894666 ...  0.          0.11627907
   0.43800055]
 [-0.31325301 -0.48768258 -0.33760972 ... -0.16666667  0.31782946
   0.52720243]
 [-0.30120482 -0.48506649 -0.19074949 ... -0.66666667  0.34883721
   0.07677437]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Testing data (X, y)
[[-0.19879518 -0.5705254  -0.37069548 ... -0.16666667  0.03875969
   0.27312897]
 [-0.21686747 -0.30237628 -0.12086428 ... -0.5         0.03875969
   0.01518917]
 [-0.04216867 -0.1942446  -0.0209318  ...  0.16666667  0.2248062
   0.04888152]
 [-0.22891566 -0.01722259 -0.13673194 ...  0.16666667 -0.31782946
   0.004971  ]
 [-0.11445783 -0.11968607  0.03511141 ...  0.         -0.054

16497/16497 [==============================] - 0s 3us/step - loss: 213.3205 - mean_squared_error: 213.3205
Epoch 53/100
16497/16497 [==============================] - 0s 3us/step - loss: 211.7289 - mean_squared_error: 211.7289
Epoch 54/100
16497/16497 [==============================] - 0s 3us/step - loss: 209.7021 - mean_squared_error: 209.7021
Epoch 55/100
16497/16497 [==============================] - 0s 3us/step - loss: 210.7580 - mean_squared_error: 210.7580
Epoch 56/100
16497/16497 [==============================] - 0s 3us/step - loss: 211.6252 - mean_squared_error: 211.6252
Epoch 57/100
16497/16497 [==============================] - 0s 3us/step - loss: 210.5877 - mean_squared_error: 210.5877
Epoch 58/100
16497/16497 [==============================] - 0s 3us/step - loss: 220.1921 - mean_squared_error: 220.1921
Epoch 59/100
16497/16497 [==============================] - 0s 3us/step - loss: 208.3233 - mean_squared_error: 208.3233
Epoch 60/100
16497/16497 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 350.4229 - mean_squared_error: 350.4229
Epoch 13/100
16498/16498 [==============================] - 0s 3us/step - loss: 312.5796 - mean_squared_error: 312.5796
Epoch 14/100
16498/16498 [==============================] - 0s 3us/step - loss: 286.8148 - mean_squared_error: 286.8148
Epoch 15/100
16498/16498 [==============================] - 0s 3us/step - loss: 270.0105 - mean_squared_error: 270.0105
Epoch 16/100
16498/16498 [==============================] - 0s 3us/step - loss: 262.8464 - mean_squared_error: 262.8464
Epoch 17/100
16498/16498 [==============================] - 0s 3us/step - loss: 253.6075 - mean_squared_error: 253.6075
Epoch 18/100
16498/16498 [==============================] - 0s 3us/step - loss: 247.0445 - mean_squared_error: 247.0445
Epoch 19/100
16498/16498 [==============================] - 0s 3us/step - loss: 243.2600 - mean_squared_error: 243.2600
Epoch 20/100
16498/16498 [===========================

...

16498/16498 [==============================] - 0s 3us/step - loss: 202.2951 - mean_squared_error: 202.2951
Epoch 92/100
16498/16498 [==============================] - 0s 3us/step - loss: 201.4346 - mean_squared_error: 201.4346
Epoch 93/100
16498/16498 [==============================] - 0s 3us/step - loss: 202.4936 - mean_squared_error: 202.4936
Epoch 94/100
16498/16498 [==============================] - 0s 3us/step - loss: 200.9383 - mean_squared_error: 200.9383
Epoch 95/100
16498/16498 [==============================] - 0s 3us/step - loss: 201.5140 - mean_squared_error: 201.5140
Epoch 96/100
16498/16498 [==============================] - 0s 3us/step - loss: 200.8825 - mean_squared_error: 200.8825
Epoch 97/100
16498/16498 [==============================] - 0s 3us/step - loss: 201.2504 - mean_squared_error: 201.2504
Epoch 98/100
16498/16498 [==============================] - 0s 3us/step - loss: 200.4947 - mean_squared_error: 200.4947
Epoch 99/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 226.3982 - mean_squared_error: 226.3982
Epoch 53/100
16498/16498 [==============================] - 0s 3us/step - loss: 226.1243 - mean_squared_error: 226.1243
Epoch 54/100
16498/16498 [==============================] - 0s 3us/step - loss: 226.7098 - mean_squared_error: 226.7098
Epoch 55/100
16498/16498 [==============================] - 0s 3us/step - loss: 224.1576 - mean_squared_error: 224.1576
Epoch 56/100
16498/16498 [==============================] - 0s 3us/step - loss: 224.1344 - mean_squared_error: 224.1344
Epoch 57/100
16498/16498 [==============================] - 0s 3us/step - loss: 223.4233 - mean_squared_error: 223.4233
Epoch 58/100
16498/16498 [==============================] - 0s 3us/step - loss: 222.0560 - mean_squared_error: 222.0560
Epoch 59/100
16498/16498 [==============================] - 0s 3us/step - loss: 223.5494 - mean_squared_error: 223.5494
Epoch 60/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 296.6312 - mean_squared_error: 296.6312
Epoch 14/100
16498/16498 [==============================] - 0s 3us/step - loss: 287.8049 - mean_squared_error: 287.8049
Epoch 15/100
16498/16498 [==============================] - 0s 3us/step - loss: 279.0704 - mean_squared_error: 279.0704
Epoch 16/100
16498/16498 [==============================] - 0s 3us/step - loss: 273.6362 - mean_squared_error: 273.6362
Epoch 17/100
16498/16498 [==============================] - 0s 3us/step - loss: 267.1976 - mean_squared_error: 267.1976
Epoch 18/100
16498/16498 [==============================] - 0s 3us/step - loss: 262.4039 - mean_squared_error: 262.4039
Epoch 19/100
16498/16498 [==============================] - 0s 3us/step - loss: 258.4564 - mean_squared_error: 258.4564
Epoch 20/100
16498/16498 [==============================] - 0s 3us/step - loss: 254.6514 - mean_squared_error: 254.6514
Epoch 21/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 203.3210 - mean_squared_error: 203.3210
Epoch 82/100
16498/16498 [==============================] - 0s 3us/step - loss: 202.5969 - mean_squared_error: 202.5969
Epoch 83/100
16498/16498 [==============================] - 0s 3us/step - loss: 202.8595 - mean_squared_error: 202.8595
Epoch 84/100
16498/16498 [==============================] - 0s 3us/step - loss: 203.1648 - mean_squared_error: 203.1648
Epoch 85/100
16498/16498 [==============================] - 0s 3us/step - loss: 201.2807 - mean_squared_error: 201.2807
Epoch 86/100
16498/16498 [==============================] - 0s 3us/step - loss: 201.0990 - mean_squared_error: 201.0990
Epoch 87/100
16498/16498 [==============================] - 0s 3us/step - loss: 200.2550 - mean_squared_error: 200.2550
Epoch 88/100
16498/16498 [==============================] - 0s 3us/step - loss: 199.2383 - mean_squared_error: 199.2383
Epoch 89/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 223.4962 - mean_squared_error: 223.4962
Epoch 43/100
16498/16498 [==============================] - 0s 3us/step - loss: 222.4376 - mean_squared_error: 222.4376
Epoch 44/100
16498/16498 [==============================] - 0s 3us/step - loss: 221.6454 - mean_squared_error: 221.6454
Epoch 45/100
16498/16498 [==============================] - 0s 3us/step - loss: 220.3564 - mean_squared_error: 220.3564
Epoch 46/100
16498/16498 [==============================] - 0s 3us/step - loss: 220.8939 - mean_squared_error: 220.8939
Epoch 47/100
16498/16498 [==============================] - 0s 3us/step - loss: 219.0195 - mean_squared_error: 219.0195
Epoch 48/100
16498/16498 [==============================] - 0s 3us/step - loss: 219.0713 - mean_squared_error: 219.0713
Epoch 49/100
16498/16498 [==============================] - 0s 3us/step - loss: 217.2953 - mean_squared_error: 217.2953
Epoch 50/100
16498/16498 [===========================

### Print the stats

In [7]:
for key in results.keys():
    
    print("\nStats for: ")
    print(key)
    evaluations_cv, evaluations_test = results[key]
    
    print("\nCrossVal stats: ")
    print(evaluations_cv)
    print(stats.describe(evaluations_cv))
    
    print("\nTest stats: ")
    print(stats.describe(evaluations_test))


Stats for: 
cmapss_alpha0.6

CrossVal stats: 
[258.60212942 215.95087132 209.04862303 240.62935567 233.41272527
 222.83101834 219.05711749 259.11277117 238.07524465 213.50122586]
DescribeResult(nobs=10, minmax=(209.04862303455477, 259.1127711669123), mean=231.0221082223048, variance=324.9684899198218, skewness=0.4542815670016322, kurtosis=-1.1105388537011844)

Test stats: 
DescribeResult(nobs=10, minmax=(204.7390835571289, 265.64044494628905), mean=223.518083404541, variance=489.78411524268176, skewness=1.0375291640068924, kurtosis=-0.41963267128960924)

Stats for: 
cmapss_alpha0.8

CrossVal stats: 
[236.37568076 261.22548605 224.19734237 224.26668423 217.57537264
 238.86054813 247.3871157  296.56490798 231.12018019 235.49571087]
DescribeResult(nobs=10, minmax=(217.57537264493536, 296.5649079834251), mean=241.30690289252158, variance=533.4776207140071, skewness=1.4461727639671123, kurtosis=1.3301390817126482)

Test stats: 
DescribeResult(nobs=10, minmax=(225.81894287109375, 285.967359